In [3]:
import os
os.getcwd()

'c:\\Users\\Admin\\Desktop\\Intelligent resume checker and job matching system\\notebooks'

In [5]:
import pandas as pd

resumes = pd.read_csv("../data/raw/resume_dataset.csv")
jobs = pd.read_csv("../data/raw/job_description_dataset.csv")

resumes.head()

,resume_id,candidate_name,email,phone,location,education,experience_years,current_role,target_role,skills,certifications,expected_salary_lpa,resume_summary
0,1,Jerry Thomas,scannon@example.net,5815367803,Bangalore,M.Tech,9,Data Analyst,AI Engineer,"AWS, Python, Kubernetes, React, C++, Git, GCP,...",NaN,27,9 years experienced professional skilled in AW...
1,2,Mr. Zachary Miller DVM,spearsmiranda@example.net,(787)477-4212,Delhi,MBA,5,Backend Developer,Cloud Engineer,"Linux, Communication, Leadership, Power BI, Ja...",NaN,3,5 years experienced professional skilled in Li...
2,3,Leah Johnson,brandon52@example.net,535.685.0007x731,Delhi,M.Tech,5,Full Stack Developer,Full Stack Developer,"CSS, Power BI, MongoDB, Teamwork, C, GCP, Seab...",AWS Certified,13,5 years experienced professional skilled in CS...
3,4,Evan Flores,merrittwilliam@example.org,(308)377-9563x2228,Delhi,M.Sc,1,Frontend Developer,Frontend Developer,"Kubernetes, Machine Learning, GitHub, Python, ...",Google Data Engineer,30,1 years experienced professional skilled in Ku...
4,5,Tammy Freeman,james27@example.org,2154384207,Bangalore,M.Sc,7,Full Stack Developer,Frontend Developer,"Linux, MySQL, Communication, PostgreSQL, R, Gi...",AWS Certified,28,7 years experienced professional skilled in Li...


In [9]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text

resumes['combined'] = (
    resumes['skills'].astype(str) + " " +
    resumes['experience_years'].astype(str) + " " +
    resumes['resume_summary'].astype(str)
)
resumes['clean_text'] = resumes['combined'].apply(clean_text)

jobs['clean_text'] = (jobs['required_skills'] + " " + jobs['job_description']).apply(clean_text)

In [10]:

resumes.head()

,resume_id,candidate_name,email,phone,location,education,experience_years,current_role,target_role,skills,certifications,expected_salary_lpa,resume_summary,combined,clean_text
0,1,Jerry Thomas,scannon@example.net,5815367803,Bangalore,M.Tech,9,Data Analyst,AI Engineer,"AWS, Python, Kubernetes, React, C++, Git, GCP,...",NaN,27,9 years experienced professional skilled in AW...,"AWS, Python, Kubernetes, React, C++, Git, GCP,...",aws python kubernetes react c git gcp html pos...
1,2,Mr. Zachary Miller DVM,spearsmiranda@example.net,(787)477-4212,Delhi,MBA,5,Backend Developer,Cloud Engineer,"Linux, Communication, Leadership, Power BI, Ja...",NaN,3,5 years experienced professional skilled in Li...,"Linux, Communication, Leadership, Power BI, Ja...",linux communication leadership power bi java r...
2,3,Leah Johnson,brandon52@example.net,535.685.0007x731,Delhi,M.Tech,5,Full Stack Developer,Full Stack Developer,"CSS, Power BI, MongoDB, Teamwork, C, GCP, Seab...",AWS Certified,13,5 years experienced professional skilled in CS...,"CSS, Power BI, MongoDB, Teamwork, C, GCP, Seab...",css power bi mongodb teamwork c gcp seaborn sq...
3,4,Evan Flores,merrittwilliam@example.org,(308)377-9563x2228,Delhi,M.Sc,1,Frontend Developer,Frontend Developer,"Kubernetes, Machine Learning, GitHub, Python, ...",Google Data Engineer,30,1 years experienced professional skilled in Ku...,"Kubernetes, Machine Learning, GitHub, Python, ...",kubernetes machine learning github python data...
4,5,Tammy Freeman,james27@example.org,2154384207,Bangalore,M.Sc,7,Full Stack Developer,Frontend Developer,"Linux, MySQL, Communication, PostgreSQL, R, Gi...",AWS Certified,28,7 years experienced professional skilled in Li...,"Linux, MySQL, Communication, PostgreSQL, R, Gi...",linux mysql communication postgresql r github ...


In [12]:
!pip install scikit-learn

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.0 MB 7.5 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/8.0 MB 7.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.0/8.0 MB 1.8 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.0 MB 1.8 MB/s eta 0:00:04
   --------- ------------------------------ 1.8/8.0 MB 1.6 MB/s eta 0:00:04
   ---------- ----------------------------- 2.1/8.0 MB 1.4 MB/s eta 0:00:05
   ----------- ---------------------------- 2.4/8.0 MB 1.6 MB/s eta 0:00:04
   ----------- ---------------------------- 2.4/8.0 MB 1.6 MB/s eta 0:00:04
   -------------- ------------------------- 2.9/8.0 MB 1.5 MB/s eta 0:00:04
   --------------- ------------------------ 3.1/8.0 MB 1.4 MB/s eta 0:00:04
   ----------------- ---------------------- 3.4/8.0 MB 1.3 MB/s eta 0:00:04
   ------------------ --------------------- 3.7/8.0 MB 1.4 MB/s eta 0:00:04
   ----------------


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english')

resume_vec = vectorizer.fit_transform(resumes['clean_text'])
job_vec = vectorizer.transform(jobs['clean_text'])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

scores = cosine_similarity(job_vec[0], resume_vec)[0]
resumes['match_score'] = scores * 100

resumes[['candidate_name', 'match_score']].sort_values(by='match_score', ascending=False).head(5)

,candidate_name,match_score
1594,Michelle King,73.275992
8530,Christopher Nolan,73.266198
2993,Alicia Sullivan,73.235941
9886,Kimberly Newton,70.874127
5660,Kelli Smith,69.454058


In [17]:
import os

os.makedirs("../data/processed", exist_ok=True)

In [19]:
resumes.sort_values(by='match_score', ascending=False)\
       .head(10)\
       .to_csv("../data/processed/top_matches.csv", index=False)